<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/data_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks')
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .

In [30]:
from batchgenerators.utilities.file_and_folder_operations import *
import shutil
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from nnunetv2.paths import nnUNet_raw


def convert_aneurysm(kits_base_dir: str, nnunet_dataset_id: int = 220):
    task_name = "aneurysm"

    foldername = "Dataset%03.0d_%s" % (nnunet_dataset_id, task_name)

    # setting up nnU-Net folders
    out_base = join('/content/drive/MyDrive/ColabNotebooks/nnUNet/nnunetv2/nnUNet_raw', foldername)
    imagestr = join(out_base, "imagesTr")
    labelstr = join(out_base, "labelsTr")
    maybe_mkdir_p(imagestr)
    maybe_mkdir_p(labelstr)

    cases = subdirs(kits_base_dir, prefix='sub_', join=False)
    for tr in cases:
        shutil.copy(join(kits_base_dir, tr, 'angio.nii.gz'), join(imagestr, f'{tr}_0000.nii.gz'))
        shutil.copy(join(kits_base_dir, tr, 'Lesion_1_mask.nii.gz'), join(labelstr, f'{tr}.nii.gz'))

    generate_dataset_json(out_base, {0: "TOF"},
                          labels={
                              "background": 0,
                              "aneurysm": 1
                          },
                          num_training_cases=len(cases), file_ending='.nii.gz')

convert_aneurysm('/content/drive/MyDrive/ColabNotebooks/data', 666)

In [28]:
!set nnUNet_raw="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_raw"
!set nnUNet_preprocessed="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_preprocessed"
!set nnUNet_results="/content/drive/MyDrive/ColabNotebooks/nnUNet/nnUNet_results"
!nnUNetv2_plan_and_preprocess -d 'Dataset666_aneurysm' --verify_dataset_integrity

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.
usage: nnUNetv2_plan_and_preprocess
       [-h]
       [-d D [D ...]]
       [-fpe FPE]
       [-npfp NPFP]
       [--verify_dataset_integrity]
       [--no_pp]
       [--clean]
       [-pl PL]
       [-gpu_memory_target GPU_MEMORY_TARGET]
       [-preprocessor_name PRE

In [ ]:
!nnUNetv2_train 'Dataset666_aneurysm' UNET_CONFIGURATION FOLD [additional options, see -h]